In [18]:
# -----------------------------------------
# Import Packages
# -----------------------------------------
import os
from datetime import datetime, timezone

import sqlite3
import pandas as pd
from sqlalchemy import create_engine
from nba_api.stats.endpoints import playergamelogs

In [20]:
# Delete G-League Data
conn = sqlite3.connect("nba_box_scores.db")
# Execute the query
cursor = conn.cursor()
cursor.execute("DELETE FROM game_details WHERE GAME_ID > 2000000000;")
cursor.execute("DELETE FROM player_stats WHERE GAME_ID > 2000000000;")
cursor.execute("DELETE FROM team_stats WHERE GAME_ID > 2000000000;")
cursor.execute("DELETE FROM inactive_players WHERE GAME_ID > 2000000000;")
cursor.execute("DELETE FROM player_basic_stats WHERE GAME_ID > 2000000000;")
cursor.execute("DELETE FROM team_basic_stats WHERE GAME_ID > 2000000000;")
conn.commit()  # Commit changes to the database
print("G League Data Deleted")



G League Data Deleted


In [22]:
# Combine team_data and team_basic_data into all_team_data
#
#
# cursor.execute("""CREATE TABLE all_team_data AS
#     SELECT DISTINCT * FROM (
#     SELECT 
#         ts.GAME_ID,
#         ts.TEAM_ID,
#         ts.TEAM_NAME,
#         ts.TEAM_ABBREVIATION,
#         ts.TEAM_CITY,
#         ts.MIN,
#         ts.E_OFF_RATING,
#         ts.OFF_RATING,
#         ts.E_DEF_RATING,
#         ts.DEF_RATING,
#         ts.E_NET_RATING,
#         ts.NET_RATING,
#         ts.AST_PCT,
#         ts.AST_TOV,
#         ts.AST_RATIO,
#         ts.OREB_PCT,
#         ts.DREB_PCT,
#         ts.REB_PCT,
#         ts.E_TM_TOV_PCT,
#         ts.TM_TOV_PCT,
#         ts.EFG_PCT AS ts_EFG_PCT,
#         ts.TS_PCT,
#         ts.USG_PCT,
#         ts.E_USG_PCT,
#         ts.E_PACE,
#         ts.PACE,
#         ts.PACE_PER40,
#         ts.POSS,
#         ts.PIE,
#         tbs.MIN AS tbs_MIN,
#         tbs.FGM,
#         tbs.FGA,
#         tbs.FG_PCT,
#         tbs.FG3M,
#         tbs.FG3A,
#         tbs.FG3_PCT,
#         tbs.FTM,
#         tbs.FTA,
#         tbs.FT_PCT,
#         tbs.OREB,
#         tbs.DREB,
#         tbs.REB,
#         tbs.AST,
#         tbs.STL,
#         tbs.BLK,
#         tbs."TO" AS tbs_TO,
#         tbs.PF,
#         tbs.PTS,
#         tbs.PLUS_MINUS
#     FROM 
#         team_stats ts
#     INNER JOIN t
#         team_basic_stats tbs
#     ON 
#         ts.GAME_ID = tbs.GAME_ID 
#         AND ts.TEAM_ID = tbs.TEAM_ID
#         AND ts.TEAM_NAME = tbs.TEAM_NAME
#         AND ts.TEAM_ABBREVIATION = tbs.TEAM_ABBREVIATION
#         AND ts.TEAM_CITY = tbs.TEAM_CITY
# 		AND ts.MIN = tbs.MIN
# ) AS combined_data;""")
# conn.commit()
# print("joined team_data")


In [24]:
# Combine player_data and player_basic_data into all_player_data
#
#
# cursor.execute("""CREATE TABLE all_player_data AS
# SELECT 
#     pbs.GAME_ID,
#     pbs.TEAM_ID,
#     pbs.TEAM_ABBREVIATION,
#     pbs.TEAM_CITY,
#     pbs.PLAYER_ID,
#     pbs.PLAYER_NAME,
#     pbs.NICKNAME,
#     pbs.START_POSITION,
#     pbs.COMMENT,
#     pbs.MIN,
#     pbs.FGM,
#     pbs.FGA,
#     pbs.FG_PCT,
#     pbs.FG3M,
#     pbs.FG3A,
#     pbs.FG3_PCT,
#     pbs.FTM,
#     pbs.FTA,
#     pbs.FT_PCT,
#     pbs.OREB,
#     pbs.DREB,
#     pbs.REB,
#     pbs.AST,
#     pbs.STL,
#     pbs.BLK,
#     pbs."TO",
#     pbs.PF,
#     pbs.PTS,
#     pbs.PLUS_MINUS,
#     ps.E_OFF_RATING,
#     ps.OFF_RATING,
#     ps.E_DEF_RATING,
#     ps.DEF_RATING,
#     ps.E_NET_RATING,
#     ps.NET_RATING,
#     ps.AST_PCT,
#     ps.AST_TOV,
#     ps.AST_RATIO,
#     ps.OREB_PCT,
#     ps.DREB_PCT,
#     ps.REB_PCT,
#     ps.TM_TOV_PCT,
#     ps.EFG_PCT,
#     ps.TS_PCT,
#     ps.USG_PCT,
#     ps.E_USG_PCT,
#     ps.E_PACE,
#     ps.PACE,
#     ps.PACE_PER40,
#     ps.POSS,
#     ps.PIE
# FROM 
#     player_basic_stats pbs
# INNER JOIN 
#     player_stats ps
# ON 
#     pbs.GAME_ID = ps.GAME_ID
#     AND pbs.TEAM_ID = ps.TEAM_ID
#     AND pbs.TEAM_ABBREVIATION = ps.TEAM_ABBREVIATION
#     AND pbs.TEAM_CITY = ps.TEAM_CITY
#     AND pbs.PLAYER_ID = ps.PLAYER_ID
#     AND pbs.PLAYER_NAME = ps.PLAYER_NAME
#     AND pbs.NICKNAME = ps.NICKNAME
#     AND pbs.START_POSITION = ps.START_POSITION
#     AND pbs.COMMENT = ps.COMMENT
# 	AND pbs.MIN = ps.MIN; """)
# conn.commit()
# conn.close()
# print("combined all player data")


In [38]:
# Combine all the player and team data with 
#
#
# cursor.execute("""CREATE TABLE combined_player_game_data AS
# SELECT 
#     apd.*,
#     gd.GAME_DATE_EST,
#     gd.GAME_SEQUENCE,
#     gd.GAME_STATUS_ID,
#     gd.GAME_STATUS_TEXT,
#     gd.GAMECODE,
#     gd.HOME_TEAM_ID,
#     gd.VISITOR_TEAM_ID,
#     gd.SEASON,
#     gd.LIVE_PERIOD,
#     gd.LIVE_PC_TIME,
#     gd.NATL_TV_BROADCASTER_ABBREVIATION,
#     gd.LIVE_PERIOD_TIME_BCAST,
#     gd.WH_STATUS
# FROM 
#     all_player_data apd
# LEFT JOIN 
#     game_details gd
# ON 
#     apd.GAME_ID = gd.GAME_ID;""")

# cursor.execute("""CREATE TABLE combined_team_game_data AS
# SELECT 
#     atd.*,
#     gd.GAME_DATE_EST,
#     gd.GAME_SEQUENCE,
#     gd.GAME_STATUS_ID,
#     gd.GAME_STATUS_TEXT,
#     gd.GAMECODE,
#     gd.HOME_TEAM_ID,
#     gd.VISITOR_TEAM_ID,
#     gd.SEASON,
#     gd.LIVE_PERIOD,
#     gd.LIVE_PC_TIME,
#     gd.NATL_TV_BROADCASTER_ABBREVIATION,
#     gd.LIVE_PERIOD_TIME_BCAST,
#     gd.WH_STATUS
# FROM 
#     all_team_data atd
# LEFT JOIN 
#     game_details gd
# ON 
#     atd.GAME_ID = gd.GAME_ID;""")
# conn.commit()
# conn.close()
# print("all data combined")

all data combined
